In [55]:
import numpy as np
import json
import gzip
from scipy.sparse import coo_matrix
from scipy.stats import binned_statistic_2d
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd

In [2]:
data_dir = '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/'
design = 'xbar'
variant = 1

In [3]:
designs_list = [
    'xbar'
]
num_designs = len(designs_list)
num_variants_list = [
    13
]

sample_names = []
corresponding_design = []
corresponding_variant = []
for idx in range(num_designs):
    for variant in range(num_variants_list[idx]):
        sample_name = data_dir + designs_list[idx] + '/' + str(variant + 1) + '/'
        sample_names.append(sample_name)
        corresponding_design.append(designs_list[idx])
        corresponding_variant.append(variant + 1)

In [4]:
sample_names

['../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/1/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/2/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/3/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/4/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/5/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/6/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/7/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/8/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/9/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/10/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/11/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/12/',
 '../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/13/']

In [5]:
corresponding_variant

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [6]:
with open(data_dir + '/settings.csv') as f:
    lines = f.readlines()

lines = lines[1:]
for line in lines:
    words = line.strip().split(',')
    dictionary = {
        'design': words[0],
        'variant': int(words[1]),
        'core_utilization': float(words[2]),
        'max_routing_layer': words[3],
        'clk_per': int(words[4]),
        'clk_uncertainty': float(words[5]),
        'flow_stage': words[6],
        'hstrap_layer': words[7],
        'hstrap_width': float(words[8]),
        'hstrap_pitch': float(words[9]),
        'vstrap_layer': words[10],
        'vstrap_width': float(words[11]),
        'vstrap_pitch': float(words[12])
    }

    if dictionary['design'] == design and dictionary['variant'] == variant:
        break

print(dictionary)

{'design': 'xbar', 'variant': 12, 'core_utilization': 0.9, 'max_routing_layer': 'MINT3', 'clk_per': 10, 'clk_uncertainty': 20.0, 'flow_stage': 'route_opt', 'hstrap_layer': 'MG1', 'hstrap_width': 0.5, 'hstrap_pitch': 4.0, 'vstrap_layer': 'MG2', 'vstrap_width': 0.5, 'vstrap_pitch': 4.0}


In [7]:
with gzip.open(data_dir + 'cells.json.gz', 'r') as f:
    cell_data = json.load(f)

widths = []
heights = []
for idx in range(len(cell_data)):
    width = cell_data[idx]['width']
    height = cell_data[idx]['height']
    widths.append(width)
    heights.append(height)

In [8]:
min_cell_width = np.min(widths)
max_cell_width = np.max(widths)
min_cell_height = np.min(heights)
max_cell_height = np.max(heights)

print('min cell width:', min_cell_width)
print('max cell width:', max_cell_width)
print('mean cell width:', np.mean(widths))
print('std cell width:', np.std(widths))
print()
print('min cell height:', min_cell_height)
print('max cell height:', max_cell_height)
print('mean cell height:', np.mean(heights))
print('std cell height:', np.std(heights))

min cell width: 256
max cell width: 397440
mean cell width: 14540.25
std cell width: 63888.21937522989

min cell height: 1536
max cell height: 503056
mean cell height: 16009.0
std cell height: 68982.92421027106


In [9]:
# Scale between 0 and 1
widths = (widths - min_cell_width) / (max_cell_width - min_cell_width)
heights = (heights - min_cell_height) / (max_cell_height - min_cell_height)
print('Done processing cell sizes')

Done processing cell sizes


In [10]:
# For each cell type writes all pins and input/output type
cell_to_edge_dict = {item['id']:{inner_item['id']: inner_item['dir'] for inner_item in item['terms']} for item in cell_data}
print('Done processing edge types dict')

Done processing edge types dict


In [ ]:
cell_to_edge_dict

NameError: name 'cell_to_edge_dict' is not defined

In [12]:
folder = sample_names[0]
with gzip.open(folder + design + '.json.gz', 'r') as fin:
    instances_nets_data = json.load(fin)

instances = instances_nets_data['instances']
nets = instances_nets_data['nets']

inst_to_cell = {item['id']:item['cell'] for item in instances}

In [13]:
num_instances = len(instances)
num_nets = len(nets)

print('Number of instances:', num_instances)
print('Number of nets:', num_nets)

Number of instances: 3952
Number of nets: 4482


In [14]:
xloc_list = [instances[idx]['xloc'] for idx in range(num_instances)]
yloc_list = [instances[idx]['yloc'] for idx in range(num_instances)]
cell = [instances[idx]['cell'] for idx in range(num_instances)]
cell_width = [widths[cell[idx]] for idx in range(num_instances)]
cell_height = [heights[cell[idx]] for idx in range(num_instances)]
orient = [instances[idx]['orient'] for idx in range(num_instances)]

In [15]:
x_min = min(xloc_list)
x_max = max(xloc_list)
y_min = min(yloc_list)
y_max = max(yloc_list)

print('min xloc:', x_min)
print('max xloc:', x_max)
print('min yloc:', y_min)
print('max yloc:', y_max)

min xloc: 512
max xloc: 84096
min yloc: 1536
max yloc: 87552


In [16]:
X = np.expand_dims(np.array(xloc_list), axis = 1)
Y = np.expand_dims(np.array(yloc_list), axis = 1)
X = (X - x_min) / (x_max - x_min)
Y = (Y - y_min) / (y_max - y_min)

In [17]:
cell = np.expand_dims(np.array(cell), axis = 1)
cell_width = np.expand_dims(np.array(cell_width), axis = 1)
cell_height = np.expand_dims(np.array(cell_height), axis = 1)
orient = np.expand_dims(np.array(orient), axis = 1)

instance_features = np.concatenate((X, Y, cell, cell_width, cell_height, orient), axis = 1)

In [18]:
dictionary = {
    'num_instances': num_instances,
    'num_nets': num_nets,
    'x_min': x_min,
    'x_max': x_max,
    'y_min': y_min,
    'y_max': y_max,
    'min_cell_width': min_cell_width,
    'max_cell_width': max_cell_width,
    'min_cell_height': min_cell_height,
    'max_cell_height': max_cell_height,
    'instance_features': instance_features,
    'sample_name': sample_name,
    'folder': sample_names[0],
    'design': design
}

In [19]:
dictionary

{'num_instances': 3952,
 'num_nets': 4482,
 'x_min': 512,
 'x_max': 84096,
 'y_min': 1536,
 'y_max': 87552,
 'min_cell_width': 256,
 'max_cell_width': 397440,
 'min_cell_height': 1536,
 'max_cell_height': 503056,
 'instance_features': array([[4.96171516e-01, 5.00000000e-01, 2.30000000e+01, 4.83403158e-03,
         0.00000000e+00, 0.00000000e+00],
        [4.96171516e-01, 5.35714286e-01, 2.30000000e+01, 4.83403158e-03,
         0.00000000e+00, 6.00000000e+00],
        [5.22205207e-01, 5.00000000e-01, 2.30000000e+01, 4.83403158e-03,
         0.00000000e+00, 0.00000000e+00],
        ...,
        [2.72588055e-01, 7.50000000e-01, 3.40000000e+01, 3.22268772e-04,
         0.00000000e+00, 0.00000000e+00],
        [4.79326187e-01, 5.00000000e-01, 1.10000000e+01, 9.66806316e-04,
         0.00000000e+00, 0.00000000e+00],
        [5.54364472e-01, 5.00000000e-01, 1.10000000e+01, 9.66806316e-04,
         0.00000000e+00, 6.00000000e+00]]),
 'sample_name': '../../data/chips/NCSU-DigIC-GraphData-2023-0

In [20]:
connection_data = np.load(sample_names[0] + design + '_connectivity.npz')

In [21]:
dirs = []
edge_t = connection_data['data']
instance_idx = connection_data['row']

for idx in range(len(instance_idx)):
    inst = instance_idx[idx]
    cell = inst_to_cell[inst]
    edge_dict = cell_to_edge_dict[cell]
    t = edge_t[idx]
    direction = edge_dict[t]
    dirs.append(direction)

dirs = np.array(dirs)

In [22]:
dictionary = {
    'instance_idx': connection_data['row'],
    'net_idx': connection_data['col'],
    'edge_attr': connection_data['data'],
    'edge_dir': dirs, 
    'sample_name': sample_name,
    'folder': sample_names[0],
    'design': design
}

In [23]:
edge_index = np.array([dictionary['instance_idx'], dictionary['net_idx']]).T
edge_dir = dictionary['edge_dir']
n_edge_index = []
for idx in range(len(edge_index)):
    tp = edge_index[idx]
    direct = edge_dir[idx]

    if direct == 0:
        n_edge_index.append([tp[1], tp[0]])
    else:
        n_edge_index.append([tp[0], tp[1]])

n_edge_index = np.array(n_edge_index).T

dictionary['edge_index'] = n_edge_index

In [ ]:
# Get source node of nets
instances = dictionary['instance_idx']
nets = dictionary['net_idx']
directs = dictionary['edge_dir']
attrs = dictionary['edge_attr']

drive_dict = dict()
for idx in range(len(instances)):
    direct = directs[idx]
    net = nets[idx]

    if direct == 1:
        assert net not in drive_dict

        drive_dict[net] = idx
    else:
        continue

In [27]:
drive_dict

{530: 2835,
 531: 2836,
 532: 2837,
 533: 2838,
 534: 2839,
 535: 2840,
 536: 2841,
 537: 2842,
 538: 2843,
 539: 2844,
 540: 2845,
 541: 2846,
 542: 2847,
 543: 2848,
 544: 2849,
 545: 2850,
 546: 2851,
 547: 2852,
 548: 2853,
 549: 2854,
 550: 2855,
 551: 2856,
 552: 2857,
 553: 2858,
 554: 2859,
 555: 2860,
 556: 2861,
 557: 2862,
 558: 2863,
 559: 2864,
 560: 2865,
 561: 2866,
 562: 2867,
 563: 2868,
 564: 2869,
 565: 2870,
 566: 2871,
 567: 2872,
 568: 2873,
 569: 2874,
 570: 2875,
 571: 2876,
 572: 2877,
 573: 2878,
 574: 2879,
 575: 2880,
 576: 2881,
 577: 2882,
 578: 2883,
 579: 2884,
 580: 2885,
 581: 2886,
 582: 2887,
 583: 2888,
 584: 2889,
 585: 2890,
 586: 2891,
 587: 2892,
 588: 2893,
 589: 2894,
 590: 2895,
 591: 2896,
 592: 2897,
 593: 2898,
 594: 2899,
 595: 2900,
 596: 2901,
 597: 2902,
 598: 2903,
 599: 2904,
 600: 2905,
 601: 2906,
 602: 2907,
 603: 2908,
 604: 2909,
 605: 2910,
 606: 2911,
 607: 2912,
 608: 2913,
 609: 2914,
 610: 2915,
 611: 2916,
 612: 2917,
 613

In [39]:
n_row = []
n_col = []
n_edge_attr = []
b_nets = []
b_terms = []
for idx in range(len(nets)):
    net = nets[idx]
    inst = instances[idx]
    attr = attrs[idx]
    
    if net not in drive_dict:
        b_nets.append(net)
        b_terms.append(idx)
    else:
        inst_idx = drive_dict[net]
        drive_inst = instances[inst_idx]
        
        if inst == drive_inst:
            continue
            
        n_row.append(drive_inst)
        n_col.append(inst)
        n_edge_attr.append(attr)

edge_index = np.array([n_row, n_col]).T
dictionary['edge_index'] = edge_index
dictionary['b_nets'] = b_nets
dictionary['b_terms'] = b_terms

In [44]:
pos_lst = instance_features[:, :2]
for idx in range(len(edge_index)):
    tp = edge_index[idx]
    direct = edge_dir[idx]

    # compute the edge distances based on nodes' positions
    first_pos = pos_lst[tp[0]]
    second_pos = pos_lst[tp[1]]
    l1_dis = np.linalg.norm((first_pos - second_pos), ord=1)
    new_attr = [l1_dis]
    if idx == 0:
        print(new_attr)
    n_edge_attr[idx] = new_attr


n_edge_attr = np.array(n_edge_attr)
print(n_edge_attr.shape, n_edge_attr[-1])
dictionary['edge_attr'] = n_edge_attr

[0.1919164296652811]
(10657, 1) [0.05819296]


In [51]:
congestion_fn = folder + design + '_congestion.npz'
congestion_data = np.load(congestion_fn)
print('Congestion info:', congestion_fn)

Congestion info: ../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_congestion.npz


In [52]:
congestion_data_demand = congestion_data['demand']
congestion_data_capacity = congestion_data['capacity']

num_layers = len(list(congestion_data['layerList']))
print('Number of layers:', num_layers)
print('Layers:', list(congestion_data['layerList']))

ybl = congestion_data['yBoundaryList']
xbl = congestion_data['xBoundaryList']

Number of layers: 13
Layers: ['M1', 'MINT1', 'MINT2', 'MINT3', 'MINT4', 'MINT5', 'MSMG1', 'MSMG2', 'MSMG3', 'MSMG4', 'MSMG5', 'MG1', 'MG2']


In [64]:
all_demand = []
all_capacity = []

for layer in list(congestion_data['layerList']):
    print('Layer', layer, ':')
    lyr = list(congestion_data['layerList']).index(layer)

    # Binned statistics 2D
    ret = binned_statistic_2d(xloc_list, yloc_list, None, 'count', bins = [xbl[1:], ybl[1:]], expand_binnumbers = True)

    i_list = np.array([ret.binnumber[0, idx] - 1 for idx in range(num_instances)])
    j_list = np.array([ret.binnumber[1, idx] - 1 for idx in range(num_instances)])

    # Get demand and capacity
    demand_list = congestion_data_demand[lyr, i_list, j_list].flatten()
    capacity_list = congestion_data_capacity[lyr, i_list, j_list].flatten()

    demand_list = np.array(demand_list)
    capacity_list = np.array(capacity_list)

    all_demand.append(np.expand_dims(demand_list, axis = 1))
    all_capacity.append(np.expand_dims(capacity_list, axis = 1))

    average_demand = np.mean(demand_list)
    average_capacity = np.mean(capacity_list)
    average_diff = np.mean(capacity_list - demand_list)
    count_congestions = np.sum(demand_list > capacity_list)

    print('    Number of demand > capacity:', count_congestions)
    print('    Average capacity - demand:', average_diff)
    print('    Average demand:', average_demand)
    print('    Average capacity:', average_capacity)

Layer M1 :
    Number of demand > capacity: 4
    Average capacity - demand: 0.013663967611336033
    Average demand: 0.013663967611336033
    Average capacity: 0.027327935222672066
Layer MINT1 :
    Number of demand > capacity: 3301
    Average capacity - demand: -2.235323886639676
    Average demand: 13.14094129554656
    Average capacity: 10.905617408906883
Layer MINT2 :
    Number of demand > capacity: 3011
    Average capacity - demand: -1.8729757085020242
    Average demand: 11.740890688259109
    Average capacity: 9.867914979757085
Layer MINT3 :
    Number of demand > capacity: 0
    Average capacity - demand: 11.73911943319838
    Average demand: 0.019230769230769232
    Average capacity: 11.758350202429149
Layer MINT4 :
    Number of demand > capacity: 0
    Average capacity - demand: 0.0
    Average demand: 0.0
    Average capacity: 0.0
Layer MINT5 :
    Number of demand > capacity: 0
    Average capacity - demand: 0.0
    Average demand: 0.0
    Average capacity: 0.0
Layer M

In [78]:
demand = np.concatenate(all_demand, axis = 1)
capacity = np.concatenate(all_capacity, axis = 1)

dictionary = {
    'demand': demand,
    'capacity': capacity
}

In [79]:
dictionary

{'demand': array([[ 0, 17, 13, ...,  0,  0,  0],
        [ 0, 14, 12, ...,  0,  0,  0],
        [ 0, 16, 14, ...,  0,  0,  0],
        ...,
        [ 0, 10,  6, ...,  0,  0,  0],
        [ 0, 13, 12, ...,  0,  0,  0],
        [ 0, 11, 12, ...,  0,  0,  0]], dtype=int64),
 'capacity': array([[ 0, 12, 12, ...,  0,  0,  0],
        [ 0, 12, 12, ...,  0,  0,  0],
        [ 0, 12, 12, ...,  0,  0,  0],
        ...,
        [ 0, 12,  4, ...,  0,  0,  0],
        [ 0, 12, 12, ...,  0,  0,  0],
        [ 0,  9, 12, ...,  0,  0,  0]], dtype=int64)}